In [ ]:
import tensorflow as tf
import pandas as pd
import os
import time


###para la inferencia podemos o espera un intervalod e tiempo para crear un eventcount_vector o sera que creamos una pequena memoria con cierta cantidad de lieas de logs hacia atras y vamos creando vectores y probando.

Creo que no tiene sentido hacer inferencia de linea en linea
 
- TODO: pensar y validar en codigo como hacer la inferencia de vainas nuevas
 

- definir cuales son los indicadores que se mostraran a dayco, tiempo de procesamiento, memoria, eficiencia delproceso

- justificar con argumentos porque el proceso funcionara con la data dayco igual que con la data de prueba

- como hacer en tiempo de inferencia, podemos simular que un archivo se va llenando de logs???revisar como cambia un archivo de logs en el logparser???

In [ ]:
'''
!apt-get update
!apt-get install -y wget bzip2
!apt-get install -y gcc perl git
!rm -rf /var/lib/apt/lists/*

!cd /
!mkdir anaconda
!cd anaconda
!wget https://repo.anaconda.com/archive/Anaconda2-5.2.0-Linux-x86_64.sh
!bash Anaconda2-5.2.0-Linux-x86_64.sh \n\n\n\n\n\n\n\n\n
!source ~/.bashrc
!cd ..
!rm -r anaconda 
'''

## clonamos loglizer

In [ ]:
#!git clone https://github.com/elieser1101/loglizer.git /loglizer/

In [ ]:
#os.chdir('/loglizer')
#!git pull origin master

## iniciamos submodulo de git

In [ ]:
#os.chdir('/loglizer')
#!git submodule update --init

In [ ]:
!pwd
!ls

## importamos sys y agregamos algunas rutas necesarias para el logparser al path de python

In [ ]:
import sys
#TODO: esto se debe incluir en repo, ver si hay una manera correcta o si es haciendo append del path!
#os.chdir('/loglizer')
repo_path = '/home/us1/git/loglizer'
sys.path.append(repo_path)#logsig __init__.py
sys.path.append(repo_path + '/logparser/logparser/LogSig')#logsig __init__.py
sys.path.append(repo_path + '/logparser/logparser/LenMa/')#for lenma __init__.py
sys.path.append(repo_path + '/logparser/logparser/LenMa/templateminer')#for lenma
from pipeline.pipeline import Pipeline
from pipeline.pipeline import Parser
from pipeline.pipeline import Loglizer
#os.chdir('/loglizer')

## mining invariants

### hack para probar el loglizer en pipeline,
luego de instanciar pipeline, cambiamos los atributos, para que apunten al dataset con el que funciona mining invariants, y creamos una nueva instancia de loglizer actualmente el `loglizer_input_dir` apunta a la salida del parser que es correcto, pero hasta que no se tenga el MiningInvariants adaptado a esa entrada se rompera
<br><br>
###NOTA: se puede jugar y obtener cualquier intancia de parser modificando los atributos de pipeline y haciendo get_parser, lo mismo para los loglizers

### visualizamos logs dayco

In [ ]:
!head {repo_path}/rocore1.log
print('******')
!head {repo_path}/rocore2.log

# Pipeline
## intentemos correr pipeline


### nota importante, es deinteres seleccionar con cuidado el parser pues, de la cantidad de eventos que este descubra dependera el gasto computacional del mining invarina


### seleccionaamos la cantidade de lineas con la que trabajaremos

In [ ]:
#!cp {repo_path}/rocore2.log {repo_path}/dayco_log.log

#!wc -l {repo_path}/dayco_log.log
#!ls -sh {repo_path}/dayco_log.log

In [ ]:
#solo par auna pruea
#!head -100 {repo_path}/rocore2.log > {repo_path}/dayco_log.log
!wc -l {repo_path}/dayco_log.log


### NOTA: es importante que la ultima linea del archivo inicial con el que trabajeroms no contenga el salto de linea
nosotros simplemente estamos copiando la ultima linea sin el salto de linea
### NOTA:cuando se utilice un dataset completo tener cuidado de no darle el tratamiento que sigue

In [ ]:

file = open(repo_path+'/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path+'/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path+'/dayco_log.log', 'r')
lines = file.readlines()
file.close()

In [ ]:
print(len(lines))
print(lines[-2:])
!wc -l {repo_path}/dayco_log.log


In [1]:
input_dir  = repo_path +'/'  # The input directory of log file
output_dir = repo_path + '/'  # The output directory of parsing results
log_file   = 'dayco_log.log'  # The input log file name
#log_format = '<Label> <Timestamp> <Date> <Node> <Time> <NodeRepeat> <Type> <Component> <Level> <Content>'#BGL
log_format = '<smonth> <sday> <shour> <ip> <id> <id2> <month> <day> <hour> <city> <type> <Content>'#dayco/rsyslog

pipeline = Pipeline(parser_algorithm='drain', input_dir = input_dir, parser_output_dir = output_dir, log_file = log_file,
parser_regex = log_format, feature_extractor='fixed_window', log_analizer_algorithm='mining_invariants',
data_type='time_based', elasticsearch_index_name='deepia')

NameError: name 'repo_path' is not defined

### en time_windows el feature extracto guarda una copia de lso indices de inicio y final de cada sliding window, por eso debemos borrar ese directorio cuando vamos a probar con archivos de diferentes tamanos(si se mantien la misma configuracion de swliding window y step size)

In [ ]:
!rm /dayco_log.log_structured.csv
!rm /dayco_log.log_templates.csv
!rm /dayco_log.logTemplateMap.csv
!rm -r /time_windows

### Logica Online

parsing
- vamos parseando cada evento que entra???
- esperamos que entre un minimo de logs???
- parsemos cada cierto tiempo???
<br> ##############<br>

que hacemos con el event count matrix??
- cada cuanto la actualizamos??
- la guardamos en csv?
- validamos la longitud y en funcion de si cmabio iniciamos otros procesos?
<br> ##############<br>

calculo de invarianes(tomar ne cuent que en general se tarda bastate esto)
- cada cuanto debemos correr encontrar invariantes??
- si sabemos que es poca data podemos correrlo cada vez que ingresa un log nuevo??
- es practico buscar invariantes con mucha periodicidad??
<br> ##############<br>

debemos hacer inferencia con log que vayan entrando
- esperamos que se tengan un minimo de eventos para armar un event count vector??
- esperamos x unidades de tiempo??? 
- o con cad log probamoss??

++++++++++++++++++++++++++++++++++++++++<br>
se me ocurre crear un metodo fisrt_time_trining, que ejecute parsing y genere una primer event_count_matrix y encuentre los primeros invariantes

pedir que se evaluen todos estos enventos

crear un metodo depia daemon, 
1.
que evalue si el logfile.log incremento su tamano o si cambio la fecha del ultimo registro, calcular cual es la diferencia de lineas(guardar cual fue la ultima linea que se parseo), ejecutar parser sobre diferencia del log
2.
sobre la diferencia del log agregar nuevo event vector, tomar en cuenta el echo de que se utiliza sliding windows, como incluyo el evento en los logs anteriores?????? se puede/tiene que actualizar la matriz???o mejor se calcula una nueva???
3. 
definimos un trshol de cantidad de logs para reiniciar el calculo de invariantes??
o mas bien el treshold debe ser temporal???
POR ULTIMO, se puede evaluar la posibilidad de ir guardando en csv un registro de invariantes???es tendria sentido????(creo que no porque me parece que los invariantes calculados solo con nuevos vectores romperan cundo se comparen con vectores viejos, ahor los viejos romperan con los nuevos??)
4.
partiendo de 1 y 2 deberiamos tener los nuevos vectores y simplemente podemos hacer una infeencia con los invriantes que se tengan para el momento1234

## corremos el proceso de generar event_count_matrix, encontrar invariantes y detectar anomalias de data original
el pipeline es capaz de correr sin etiquetas, con los metodos deepia


In [ ]:
!rm {repo_path}/time_windows/sliding_3h_1h.csv

In [ ]:
#loglizer params for BGL
para = {
'path':repo_path + '/',                 # directory for input data
'log_file_name':'dayco_log.log',           # filename for log data file
'log_event_mapping':'dayco_log.logTemplateMap.csv',   # filename for log-event mapping. A list of event index, where each row represents a log
'save_path': './time_windows/',             # dir for saving sliding window data files to avoid splitting
#'select_column':[0,4],                      # select the corresponding columns (label and time) in the raw log file
'select_column':[0,1,2],                      # select the corresponding columns (label and time) in the raw log file, en caso de depia no nos intersa la columna label
'window_size':3, #3                           # time window (unit: hour)
'step_size': 1,                             # step size (unit: hour)
'epsilon':2.0,                              # threshold for the step of estimating invariant space
'threshold':0.97,                           # percentage of vector Xj in matrix satisfies the condition that |Xj*Vi|<epsilon
'scale_list':[1,2,3],					    # list used to sacle the theta of float into integer
'stop_invar_num':3                          # stop if the invariant length is larger than stop_invar_num. None if not given
}
start = time.time()
#event_count_matrix = pipeline.log_analizer.get_event_count_matrix(para)
#invar_dict = pipeline.log_analizer.find_invariants(para, event_count_matrix)
#predictions, anomalies = pipeline.log_analizer.get_anomalies(para, event_count_matrix, invar_dict)
pipeline.initial_go(para)
end = time.time()
print('total execution time in seconds?',end - start)

In [ ]:
pipeline.run_online(para, 600, 30, 10)